In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy import stats
import sklearn
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import seaborn as sns

In [2]:
def join_two_datasets(df, other_df, fields):

    df['fips'] = df['fips'].astype(int)
    other_df['fips'] = other_df['fips'].astype(int)
    return pd.merge(df, other_df, on=fields, how='inner')

def read_csv(df_name):
    return pd.read_csv(df_name + '_processed.csv')

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
import plotly.io as pio
init_notebook_mode(connected=True)
print(1)

In [ ]:
df_chemicals = read_csv('chemicals')
df_chemicals.head()

In [ ]:
def create_plotly_map(fips, vals, filename, display=False):
    fips = list(fips)
    vals = list(vals)
    fig = ff.create_choropleth(fips=fips, values=vals)
    print('created figure')
    pio.write_image(fig, filename)
    print('writing figure')
    if display:
        print('drawing figure')
        iplot(fig, filename=filename)
    print('done')

In [ ]:
#df_chemicals_new = df_chemicals.drop(columns=['unit_measurement', 'state', 'pws_id', 'county', 'cws_name', 'state_fips'])
#df_chemicals_new.head()

In [ ]:
fips_ids = []
contaminant_levels = []
population_served = []
df_chemicals = df_chemicals.dropna()
df_chemicals = df_chemicals[df_chemicals['year'] == 2016]
df_chemicals.pop_served = df_chemicals.pop_served.astype(int)
for region, df_region in df_chemicals.groupby('fips'):
    fips_ids.append(region)
    contaminant_levels.append(sum(df_region.value))
    population_served.append(sum(df_region.pop_served))
print(len(fips_ids))
print(len(contaminant_levels))
print(len(population_served))


In [ ]:
create_plotly_map(fips_ids, population_served, 'population_map_2016.png')

In [ ]:
create_plotly_map(fips_ids, contaminant_levels, 'contaminant_map_2016.png')

In [ ]:
print(sum(df_chemicals['state_fips'] == 48))

In [ ]:
df_chemicals = read_csv('earnings')
fips_ids = []
median_earnings = []
df_chemicals = df_chemicals.dropna()
df_chemicals = df_chemicals[df_chemicals['year'] == 2016]
for region, df_region in df_chemicals.groupby('fips'):
    fips_ids.append(region)
    median_earnings.append(sum(df_region.total_med))
df_chemicals.head()
create_plotly_map(fips_ids, median_earnings, 'earnings_map_2016.png')

In [ ]:
df_chemicals = read_csv('chemicals')
df_chemicals = df_chemicals[df_chemicals['chemical_species'] == 'Nitrates']
print(len(df_chemicals))
print(len(df_chemicals.fips.unique()))
df_earnings = read_csv('earnings')
df_industry = read_csv('industry_occupation')

In [ ]:
average_by_fips = df_earnings.groupby("fips")
fips = []
yrs = []
avg_values = []

for g in average_by_fips:
    fips.append(g[0])
    avg_values.append(np.mean(g[1]["total_med"]))
fip_to_earnings = dict(zip(fips, avg_values))

In [ ]:
average_by_fips = df_chemicals.groupby("fips")
fips = []
yrs = []
avg_values = []

for g in average_by_fips:
    fips.append(g[0])
    avg_values.append(np.mean(g[1]["value"]))
fip_to_contamination = dict(zip(fips, avg_values))

In [ ]:
contamination_to_earnings = {}
contam = []
earn = []
for fip in fip_to_earnings:
    if fip in fip_to_contamination:
        contam.append(fip_to_contamination[fip])
        earn.append(fip_to_earnings[fip])
        contamination_to_earnings[fip_to_contamination[fip]] = fip_to_earnings[fip]
print(len(contam))
print(len(earn))

In [ ]:
stats.pearsonr(contam, earn)

In [3]:
df_industry = read_csv('industry_occupation')

In [4]:
df_industry.columns

Index(['geo_id', 'fips', 'county', 'total_employed', 'agriculture',
       'construction', 'manufacturing', 'wholesale_trade', 'retail_trade',
       'transport_utilities', 'information', 'finance_insurance_realestate',
       'prof_scientific_waste', 'edu_health', 'arts_recreation', 'other',
       'public_admin', 'year'],
      dtype='object')

In [5]:
fips_to_agro = {}
df_industry = df_industry.dropna(subset=['agriculture'])
df_industry.head()



,geo_id,fips,county,total_employed,agriculture,construction,manufacturing,wholesale_trade,retail_trade,transport_utilities,information,finance_insurance_realestate,prof_scientific_waste,edu_health,arts_recreation,other,public_admin,year
0,0500000US01003,1003,"Baldwin County, Alabama",77263.0,1302.0,6489.0,6134.0,2131.0,9525.0,3557.0,940.0,5893.0,9873.0,14753.0,8257.0,3575.0,4834.0,2010
1,0500000US01015,1015,"Calhoun County, Alabama",42768.0,337.0,1371.0,7868.0,750.0,5706.0,1626.0,779.0,2234.0,3776.0,8351.0,3799.0,2440.0,3731.0,2010
6,0500000US01069,1069,"Houston County, Alabama",45470.0,790.0,2967.0,4237.0,1897.0,6225.0,2838.0,670.0,1980.0,3458.0,9843.0,5200.0,2658.0,2707.0,2010
7,0500000US01073,1073,"Jefferson County, Alabama",289454.0,1761.0,17682.0,27091.0,8859.0,34024.0,14727.0,8309.0,25021.0,29164.0,67630.0,26357.0,15828.0,13001.0,2010
8,0500000US01077,1077,"Lauderdale County, Alabama",39581.0,946.0,3579.0,4285.0,1364.0,6047.0,2474.0,439.0,1771.0,2710.0,8624.0,3551.0,3148.0,643.0,2010


In [6]:
fips_names = list(df_industry.fips)
agro_names = list(df_industry.agriculture)
for f, a in zip(fips_names, agro_names):
    fips_to_agro[f] = a

In [7]:
df_chemicals = read_csv('chemicals')
df_chemicals = df_chemicals[df_chemicals['chemical_species'] == 'Nitrates']
df_chemicals.dropna()
average_by_fips = df_chemicals.groupby("fips")
fips_to_contam = {}
for g in average_by_fips:
    fips_to_contam[g[0]] = np.mean(g[1]["value"])

In [8]:

contam = []
agro = []
for fip in fips_to_contam:
    if fip in fips_to_agro:
        contam.append(fips_to_contam[fip])
        agro.append(fips_to_agro[fip])
print(contam)
print(agro)
stats.pearsonr(contam, agro)

[1225.8490566037735, 1328.4028776978416, 1540.7628865979382, 389.0, 2267.772466539197, 822.4603174603175, 775.0909090909091, 2828.9590202543573, 529.8979591836735, 1787.1815718157181, 1309.1601866251945, 1026.3057324840765, 678.3262711864406, 3019.135802469136, 3114.295719844358, 657.5945017182131, 452.3478260869565, 1367.371794871795, 368.1372549019608, 2172.5050709939146, 1384.9065420560748, 2110.2448979591836, 1694.6320346320347, 198.955223880597, 2400.4065040650407, 1899.6184210526317, 610.079575596817, 1726.848, 2769.6559633027523, 1042.837837837838, 991.8712574850299, 949.84375, 1315.0705270972533, 3403.850855745721, 4271.789473684211, 5103.303085299455, 2111.625, 3489.468085106383, 712.6422764227642, 941.5486725663717, 348.94329896907215, 35.55555555555556, 101.11111111111111, 230.87570621468927, 1300.028409090909, 992.5133689839572, 552.9591836734694, 344.8292682926829, 1202.2926829268292, 1344.2173913043478, 1783.7142857142858, 1694.4695898161244, 1193.6963696369637, 954.93167

(0.31907991602105085, 2.827167976404234e-11)

In [ ]:
import seaborn as sns

In [ ]:
plt.figure()
plt.title('Relation between Agriculture Workforce and Nitrate Contamination')
plt.scatter(agro, contam)
plt.xlabel('Agriculture Workforce P')
plt.savefig('chemicals_workforce.png')